## 1 如何确定 .pt 的输入 / 输出形状

### 1.1 对 YOLO/Ultralytics 系列模型，典型情况:

- 输入通常是：[batch, channel, height, width] = [N, 3, H, W]
- 默认模型训练/推理尺寸常见为：640x640 或 640x352/416/512 等
- 但具体还要看：
    - 模型配置里的 imgsz/img_size
    - 导出时你指定的 imgsz 参数

In [5]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')  # 换成你的.pt
# print(model.model)          # 打印网络结构
print(model.model.yaml)     # 配置（可能包含 img_size / stride 等）

{'nc': 80, 'scales': {'n': [0.5, 0.25, 1024], 's': [0.5, 0.5, 1024], 'm': [0.5, 1.0, 512], 'l': [1.0, 1.0, 512], 'x': [1.0, 1.5, 512]}, 'backbone': [[-1, 1, 'Conv', [64, 3, 2]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 2, 'C3k2', [256, False, 0.25]], [-1, 1, 'Conv', [256, 3, 2]], [-1, 2, 'C3k2', [512, False, 0.25]], [-1, 1, 'Conv', [512, 3, 2]], [-1, 2, 'C3k2', [512, True]], [-1, 1, 'Conv', [1024, 3, 2]], [-1, 2, 'C3k2', [1024, True]], [-1, 1, 'SPPF', [1024, 5]], [-1, 2, 'C2PSA', [1024]]], 'head': [[-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 6], 1, 'Concat', [1]], [-1, 2, 'C3k2', [512, False]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 4], 1, 'Concat', [1]], [-1, 2, 'C3k2', [256, False]], [-1, 1, 'Conv', [256, 3, 2]], [[-1, 13], 1, 'Concat', [1]], [-1, 2, 'C3k2', [512, False]], [-1, 1, 'Conv', [512, 3, 2]], [[-1, 10], 1, 'Concat', [1]], [-1, 2, 'C3k2', [1024, True]], [[16, 19, 22], 1, 'Detect', ['nc']]], 'scale': 'n', 'yaml_file': 'yolo11n.yaml', 'ch': 3}


同时你可以用一个 dummy 输入看一下:  
如果报错，说明分辨率不适配，可以查看 model.model.stride（如 32），确保输入尺寸是 stride 的整数倍。

In [7]:
import torch

dummy = torch.randn(1, 3, 640, 640)  # 猜一个常见尺寸先
out = model.model(dummy)
for i, o in enumerate(out if isinstance(out, (list, tuple)) else [out]):
    print(i, o.shape)

0 torch.Size([1, 84, 8400])


AttributeError: 'list' object has no attribute 'shape'

### 1.2 输出形状  
`Ultralytics YOLO` 的输出在**训练模式**和**推理模式**不太一样：  

- 训练模式（`model.model.train()`）：  
    - 通常返回多个特征层或 `loss` 相关信息，不适合作为导出目标。  
- 推理模式（`model.model.eval()` / `YOLO(...)(img)`）:  
    - 一般为 `检测 head 输出`，比如：  
        - `shape` 类似 `[batch, num_anchors * grid_x * grid_y, (x,y,w,h,conf,num_classes)]`  
        - 或者是多个 stride 层输出的 list / tuple  
对 `Ultralytics YOLOv8` `model.export(format="onnx")` 导出后，典型输出为：  
  
单个输出节点：`(batch, num_predictions, 85)`  
4 个 `bbox` 坐标  
1 个 `objectness`  
80 个类概率（COCO）  
要确认你这一个 `.pt` 的实际输出:  

你也可以直接看导出的 `ONNX`，用 `netron.app` 打开，查看每个输出节点的 shape 和含义。

In [ ]:
model = YOLO('your_model.pt')
model.model.eval()
dummy = torch.randn(1, 3, 640, 640)
with torch.no_grad():
    out = model.model(dummy)

if isinstance(out, (list, tuple)):
    for i, o in enumerate(out):
        print(i, type(o), getattr(o, 'shape', None))
else:
    print(type(out), getattr(out, 'shape', None))